In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
from datetime import datetime
from scipy.stats import linregress
from citipy import citipy



In [4]:
visitor_path = "Data/changes-visitors-covid.csv"
covid_path = "Data/covid-data.csv"
debt_path = "Data/debt-relief-covid.csv"
income_path = "Data/income-support-covid.csv"
face_path = "Data/face-covering-policies-covid.csv"
travel_path = "Data/international-travel-covid.csv"
school_path = "Data/school-closures-covid.csv"


In [11]:
visitor_data = pd.read_csv(visitor_path)
covid_data = pd.read_csv(covid_path)
debt_data = pd.read_csv(debt_path)
income_data = pd.read_csv(income_path)
face_data = pd.read_csv(face_path)
travel_data = pd.read_csv(travel_path)
school_data = pd.read_csv(school_path)


In [12]:
visitor_data.head()

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,Afghanistan,AFG,2020-02-17,0.00,4.000,1.333,5.667,1.000,-6.333
1,Afghanistan,AFG,2020-02-18,0.75,4.250,0.750,6.500,1.500,-3.250
2,Afghanistan,AFG,2020-02-19,0.40,4.000,0.800,5.200,1.400,-1.600
3,Afghanistan,AFG,2020-02-20,0.00,3.833,0.833,5.000,1.000,-0.333
4,Afghanistan,AFG,2020-02-21,0.00,4.571,0.714,5.286,1.429,0.571


In [135]:
covid_data.location.nunique()

230

In [136]:
covid_data.location.unique().tolist()

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'European Union',


In [137]:
continent = ['North America','South America','Africa','Asia','Europe']

In [140]:
# removing continents from data frame
covid_data[~covid_data.location.isin(continent)]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96203,ZWE,Africa,Zimbabwe,2021-06-12,39852.0,164.0,97.714,1632.0,3.0,3.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96204,ZWE,Africa,Zimbabwe,2021-06-13,39959.0,107.0,110.000,1632.0,0.0,3.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96205,ZWE,Africa,Zimbabwe,2021-06-14,40077.0,118.0,119.857,1635.0,3.0,3.429,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96206,ZWE,Africa,Zimbabwe,2021-06-15,40318.0,241.0,142.429,1637.0,2.0,2.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


In [141]:
covid_data.info

<bound method DataFrame.info of       iso_code continent     location        date  total_cases  new_cases  \
0          AFG      Asia  Afghanistan  2020-02-24          1.0        1.0   
1          AFG      Asia  Afghanistan  2020-02-25          1.0        0.0   
2          AFG      Asia  Afghanistan  2020-02-26          1.0        0.0   
3          AFG      Asia  Afghanistan  2020-02-27          1.0        0.0   
4          AFG      Asia  Afghanistan  2020-02-28          1.0        0.0   
...        ...       ...          ...         ...          ...        ...   
96203      ZWE    Africa     Zimbabwe  2021-06-12      39852.0      164.0   
96204      ZWE    Africa     Zimbabwe  2021-06-13      39959.0      107.0   
96205      ZWE    Africa     Zimbabwe  2021-06-14      40077.0      118.0   
96206      ZWE    Africa     Zimbabwe  2021-06-15      40318.0      241.0   
96207      ZWE    Africa     Zimbabwe  2021-06-16      40556.0      238.0   

       new_cases_smoothed  total_deaths  ne

In [110]:
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [124]:
covid_data.head()
covid_df = round(covid_data.groupby(["location"])["total_cases"].max(),2)
# type(covid_df)
covid_df.dropna(inplace=True)
covid_df

location
Afghanistan        93288.0
Africa           5101570.0
Albania           132476.0
Algeria           134458.0
Andorra            13836.0
                  ...     
Vietnam            11794.0
World          177040182.0
Yemen               6867.0
Zambia            118850.0
Zimbabwe           40556.0
Name: total_cases, Length: 201, dtype: float64

In [130]:
afg_data = covid_data[covid_data.location == "Afghanistan"].describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
count,479.000000,479.000000,474.000000,452.000000,452.000000,474.000000,479.000000,479.000000,474.000000,452.000000,...,0.0,4.790000e+02,4.790000e+02,0.0,0.0,4.790000e+02,479.0,4.790000e+02,4.790000e+02,0.0
mean,38627.185804,194.755741,189.940300,1610.030973,8.148230,7.450859,992.263860,5.002954,4.879247,41.358856,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
std,21882.369173,302.593270,273.168902,932.423675,11.198577,8.843587,562.119206,7.773092,7.017222,23.952284,...,NaN,1.820891e-12,6.401570e-14,NaN,NaN,4.054328e-13,0.0,7.966399e-13,2.889598e-15,NaN
min,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.026000,0.000000,0.000000,0.026000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
25%,29309.000000,28.500000,39.785500,1033.000000,1.000000,1.714000,752.896500,0.732000,1.022000,26.536000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
50%,40369.000000,78.000000,88.571500,1545.500000,4.000000,4.286000,1037.008000,2.004000,2.275500,39.701500,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
75%,55529.000000,214.500000,208.321000,2435.250000,11.000000,9.964250,1426.441500,5.510500,5.351500,62.557500,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
max,93288.000000,1842.000000,1584.000000,3683.000000,85.000000,61.714000,2396.403000,47.318000,40.690000,94.610000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN


In [167]:
afg_data = covid_data[covid_data.location == "Afghanistan"]
afg_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,AFG,Asia,Afghanistan,2021-06-12,88740.0,1024.0,1539.571,3449.0,37.0,49.286,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
475,AFG,Asia,Afghanistan,2021-06-13,89861.0,1121.0,1519.571,3527.0,78.0,54.571,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
476,AFG,Asia,Afghanistan,2021-06-14,91458.0,1597.0,1516.714,3612.0,85.0,60.714,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
477,AFG,Asia,Afghanistan,2021-06-15,93272.0,1814.0,1563.714,3683.0,71.0,61.714,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [169]:
afg_data['month_year'] = pd.to_datetime(afg_data['date']).dt.to_period('M')
afg_data.head()

<ipython-input-169-fb8835b80fff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afg_data['month_year'] = pd.to_datetime(afg_data['date']).dt.to_period('M')


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality,month_year
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02


In [186]:
afg_df = afg_data.groupby(["month_year"])[["location","total_cases","new_cases","month_year"]].max()
afg_df

,location,total_cases,new_cases,month_year
month_year,,,,
2020-02,Afghanistan,1.0,1.0,2020-02
2020-03,Afghanistan,175.0,33.0,2020-03
2020-04,Afghanistan,2127.0,187.0,2020-04
2020-05,Afghanistan,15208.0,866.0,2020-05
2020-06,Afghanistan,31507.0,915.0,2020-06
2020-07,Afghanistan,36665.0,348.0,2020-07
2020-08,Afghanistan,38159.0,108.0,2020-08
2020-09,Afghanistan,39268.0,125.0,2020-09
2020-10,Afghanistan,41425.0,141.0,2020-10


In [214]:
afg_df.reset_index(drop=True, inplace=True)
afg_df

,location,total_cases,new_cases,month_year
0,Afghanistan,1.0,1.0,2020-02
1,Afghanistan,175.0,33.0,2020-03
2,Afghanistan,2127.0,187.0,2020-04
3,Afghanistan,15208.0,866.0,2020-05
4,Afghanistan,31507.0,915.0,2020-06
5,Afghanistan,36665.0,348.0,2020-07
6,Afghanistan,38159.0,108.0,2020-08
7,Afghanistan,39268.0,125.0,2020-09
8,Afghanistan,41425.0,141.0,2020-10
9,Afghanistan,46274.0,282.0,2020-11


In [216]:
total_new_cases= afg_data.groupby(["month_year"])["new_cases"].sum()
total_new_cases

month_year
2020-02        1.0
2020-03      174.0
2020-04     1952.0
2020-05    13081.0
2020-06    16299.0
2020-07     5158.0
2020-08     1494.0
2020-09     1109.0
2020-10     2157.0
2020-11     4849.0
2020-12     5252.0
2021-01     3497.0
2021-02      691.0
2021-03      740.0
2021-04     3291.0
2021-05    12093.0
2021-06    21450.0
Freq: M, Name: new_cases, dtype: float64

In [217]:
total_death= afg_data.groupby(["month_year"])["total_deaths"].sum()
total_death

month_year
2020-02        0.0
2020-03       27.0
2020-04      890.0
2020-05     5007.0
2020-06    15016.0
2020-07    33518.0
2020-08    42101.0
2020-09    42987.0
2020-10    46287.0
2020-11    49133.0
2020-12    62357.0
2021-01    71838.0
2021-02    67864.0
2021-03    76227.0
2021-04    76300.0
2021-05    85386.0
2021-06    52796.0
Freq: M, Name: total_deaths, dtype: float64

In [218]:
merge_data = pd.DataFrame({"total_death":total_death, "total_new_cases": total_new_cases})
merge_data


,total_death,total_new_cases
month_year,,
2020-02,0.0,1.0
2020-03,27.0,174.0
2020-04,890.0,1952.0
2020-05,5007.0,13081.0
2020-06,15016.0,16299.0
2020-07,33518.0,5158.0
2020-08,42101.0,1494.0
2020-09,42987.0,1109.0
2020-10,46287.0,2157.0


In [219]:
month_year = afg_df["month_year"]
afg_df

,location,total_cases,new_cases,month_year
0,Afghanistan,1.0,1.0,2020-02
1,Afghanistan,175.0,33.0,2020-03
2,Afghanistan,2127.0,187.0,2020-04
3,Afghanistan,15208.0,866.0,2020-05
4,Afghanistan,31507.0,915.0,2020-06
5,Afghanistan,36665.0,348.0,2020-07
6,Afghanistan,38159.0,108.0,2020-08
7,Afghanistan,39268.0,125.0,2020-09
8,Afghanistan,41425.0,141.0,2020-10
9,Afghanistan,46274.0,282.0,2020-11


In [223]:
merge_data["month_year"] = month_year
merge_data.reset_index(drop=True, inplace=True)

In [234]:
merge_afg = pd.merge(afg_df,merge_data, on = "month_year")
merge_afg


,location,total_cases,new_cases,month_year,total_death,total_new_cases
0,Afghanistan,1.0,1.0,2020-02,0.0,1.0
1,Afghanistan,175.0,33.0,2020-03,27.0,174.0
2,Afghanistan,2127.0,187.0,2020-04,890.0,1952.0
3,Afghanistan,15208.0,866.0,2020-05,5007.0,13081.0
4,Afghanistan,31507.0,915.0,2020-06,15016.0,16299.0
5,Afghanistan,36665.0,348.0,2020-07,33518.0,5158.0
6,Afghanistan,38159.0,108.0,2020-08,42101.0,1494.0
7,Afghanistan,39268.0,125.0,2020-09,42987.0,1109.0
8,Afghanistan,41425.0,141.0,2020-10,46287.0,2157.0
9,Afghanistan,46274.0,282.0,2020-11,49133.0,4849.0


In [241]:
!pip install plotly.express

     |████████████████████████████████| 19.1 MB 6.9 MB/s eta 0:00:01


In [246]:
merge_afg.dtypes

location              object
total_cases          float64
new_cases            float64
month_year         period[M]
total_death          float64
total_new_cases      float64
dtype: object

In [253]:
merge_afg["month_year_str"] = merge_afg.month_year.astype('str')

In [254]:
# use color as third subset for countries
import plotly.express as px
# df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(merge_afg, x="month_year_str", y="total_new_cases")
fig.show()

In [134]:
covid_data[covid_data.total_cases == 93288.000000]


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
478,AFG,Asia,Afghanistan,2021-06-16,93288.0,16.0,1319.714,3683.0,0.0,54.0,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
15685,CAN,North America,Canada,2020-06-01,93288.0,809.0,881.286,7952.0,46.0,76.0,...,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929,NaN


In [99]:
len(covid_data)

96208

In [49]:
debt_data.head()

,Entity,Code,Day,debt_relief
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [83]:
income_data.head()


,Entity,Code,Day,income_support
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [259]:
income_afg = income_data[income_data.Entity == "Afghanistan"]
income_afg

,Entity,Code,Day,income_support
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0
...,...,...,...,...
508,Afghanistan,AFG,2021-06-09,0
509,Afghanistan,AFG,2021-06-10,0
510,Afghanistan,AFG,2021-06-11,0
511,Afghanistan,AFG,2021-06-12,0


In [261]:
income_afg['month_year'] = pd.to_datetime(income_afg['Day']).dt.to_period('M')
income_afg.head()

<ipython-input-261-c8bc48e76437>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Entity,Code,Day,income_support,month_year
0,Afghanistan,AFG,2020-01-01,0,2020-01
1,Afghanistan,AFG,2020-01-02,0,2020-01
2,Afghanistan,AFG,2020-01-03,0,2020-01
3,Afghanistan,AFG,2020-01-04,0,2020-01
4,Afghanistan,AFG,2020-01-05,0,2020-01


In [271]:
income_df = income_afg.groupby(["month_year"])[["Entity","income_support","month_year"]].max()

income_df.reset_index(drop=True, inplace=True)
income_df

,Entity,income_support,month_year
0,Afghanistan,0,2020-01
1,Afghanistan,0,2020-02
2,Afghanistan,0,2020-03
3,Afghanistan,0,2020-04
4,Afghanistan,0,2020-05
5,Afghanistan,0,2020-06
6,Afghanistan,0,2020-07
7,Afghanistan,0,2020-08
8,Afghanistan,0,2020-09
9,Afghanistan,0,2020-10


In [281]:
agf_2_income = pd.merge(merge_afg,income_df, on = "month_year")
agf_2_income.drop(['Entity','month_year'], axis=1)

,location,total_cases,new_cases,total_death,total_new_cases,month_year_str,income_support
0,Afghanistan,1.0,1.0,0.0,1.0,2020-02,0
1,Afghanistan,175.0,33.0,27.0,174.0,2020-03,0
2,Afghanistan,2127.0,187.0,890.0,1952.0,2020-04,0
3,Afghanistan,15208.0,866.0,5007.0,13081.0,2020-05,0
4,Afghanistan,31507.0,915.0,15016.0,16299.0,2020-06,0
5,Afghanistan,36665.0,348.0,33518.0,5158.0,2020-07,0
6,Afghanistan,38159.0,108.0,42101.0,1494.0,2020-08,0
7,Afghanistan,39268.0,125.0,42987.0,1109.0,2020-09,0
8,Afghanistan,41425.0,141.0,46287.0,2157.0,2020-10,0
9,Afghanistan,46274.0,282.0,49133.0,4849.0,2020-11,0


In [263]:
# income_afg["month_year_str"] = income_afg.month_year.astype('str')

In [ ]:
# use color as third subset for countries
import plotly.express as px
# df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(merge_afg, x="month_year_str", y="total_new_cases")
fig.show()

In [256]:

# income_df = round(income_data.groupby(["Entity"])["income_support"].mean(),2)
# income_df
# # income_df.sort_values(ascending = False)

In [270]:
# df = income_data.set_index("Entity")

# # df = income_data.loc["Afghanistan"]
# df_income = df.loc["Bulgaria"]
# df_income["income_support"].sort_values(ascending = False)

In [267]:
# face_data.head()

In [268]:
# travel_data.head()

In [269]:
# school_data.head()
